In [2]:
%matplotlib inline 
import itertools
from bs4 import BeautifulSoup
import urllib2
import requests
import pandas as pd
import re
import time
import numpy as np
import json
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
import matplotlib.pyplot as plt
from pyquery import PyQuery as pq
dropbox = "C:\Users\mkkes_000\Dropbox\Indiastuff\OutputTables"

#This notebook takes all the candidates that appear in regional elections in 2012-2015 and takes their personal info #

In [3]:
#First load the dataframe
df_fin = pd.read_csv("df_all_regional_elections2.csv")
print df_fin.shape
df_fin.head()
df_fin.columns

(37340, 29)


C:\Users\mkkes_000\Anaconda\lib\site-packages\pandas\io\parsers.py:1170: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


Index([u'Unnamed: 0', u'Age', u'Criminal Cases', u'Education Level', u'Gender',
       u'IPC Details*', u'Immovable Assets', u'Movable Assets', u'Name',
       u'Number of Cases', u'PAN Given(Y or N)', u'Party Code',
       u'Serious IPC Counts', u'Total Assets', u'Total Liabilities', u'c',
       u'consistuency', u'district', u'election', u'id', u'link', u'win',
       u'assets new', u'assets old', u'increase', u'name2', u'new_id',
       u'old_id', u'pct change'],
      dtype='object')

In [22]:
def url_tsform(el,c_id):
    url_link = "http://myneta.info/%s/candidate.php?candidate_id=%s" %(el,c_id)
    return url_link
def url_split(url_link):
    return url_link.split("/")[3], url_link.split("=")[1]

In [35]:
url_split("http://myneta.info/mp2013/candidate.php?candidate_id=666")

('mp2013', '666')

In [31]:
def get_page(el,c_id):
    # Check if URL has already been visited.
    url = (el,c_id)
    url_error = []
    if (url not in urlcache) or (urlcache[url]==1) or (urlcache[url]==2):
        time.sleep(.5)
        try:
            r = requests.get(url_tsform(el,c_id))
            if r.status_code == 200:
                urlcache[url] = r.text
            else:
                urlcache[url] = 1
        except:
            urlcache[url] = 2
            url_error.append(url)
            print "error with:", url
    return urlcache[url]

In [26]:
#Re open the dict
with open('tempdata/largedict.json', 'r') as f:
     dict_start = json.load(f)

In [75]:
dict_id = {url_split(k):v for k,v in dict_start.iteritems()}
print len(dict_id)
urlcache = {}

20998


In [78]:
#Creates a giant dictionary
#dict_id2 = {}
steps = len(dict_id2)
print "initial length: ", steps
for row in df_fin.itertuples():
    if (row[19],row[20]) in dict_id2:
        pass
    else:
        dict_id2[(row[19],row[20])] = get_page(row[19],row[20])
        steps = len(dict_id2)
        if steps % 1000 ==0:
            print steps,

initial length:  0
1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 37000


In [88]:
dict_clean = {url_tsform(k[0],k[1]):v for k,v in dict_id2.iteritems() if v!=2}
print len(dict_clean), len(dict_id2)

37340 37340


In [89]:
with open('tempdata/largedict.json', 'w') as f:
     json.dump(dict_clean, f)

In [90]:
listerror=[]
for k,v in dict_id2.iteritems():
    if isinstance(v, basestring):
        listerror.append(v)
len(listerror)

37340

In [103]:
page_ex = dict_clean['http://myneta.info/mp2013/candidate.php?candidate_id=258']

In [108]:
bs = BeautifulSoup(page_ex,"html.parser")
bs.findAll("table")[0]

<table cellpadding="0" cellspacing="0" id="table3" width="80%">
<tr style="background-color:#336633;color:white;" valign="top"><th colspan="3">Other Elections</th></tr>
<tr style="background-color:#336633;color:white;" valign="top"><th>Declaration in</th><th>Declared Assets</th><th>Declared Cases</th></tr>
<tr><td style="background-color:#F3F3F3;"><b>Loksabha 2014</b></td> <td style="background-color:#F3F3F3;"><b>Rs25,48,76,883</b><span style="color:black;text-decoration:none;white-space:nowrap"><br>~25 Crore+</br></span></td><td style="background-color:#F3F3F3;">2</td>
</tr>
<tr><td colspan="3"><a href="/compare_profile.php?group_id=uRKRoKnCXyy5tSAuuc8b">Click here for more details</a></td></tr></table>

In [111]:
ROOT_LINK = "http://myneta.info"
def get_otherelec_link(page_text, url):
    for a in pq(page_text)("a"):
        if a.text == "Click here for more details":
            other_elec_link = pq(a).attr.href
            return ROOT_LINK+other_elec_link
    return False

test_index = "http://myneta.info/mp2013/candidate.php?candidate_id=258"
page_data = dict_clean[test_index]
other_link = get_otherelec_link(page_data,test_index)

In [112]:
def get_otherelec_data(otherelec_link):
    
    otherelec_dict = {'common_link': otherelec_link}
    
    html = requests.get(otherelec_link)
    doc = pq(html.content)
    
    columns = []
    all_dicts = []
    add = 0
    trs = doc('tr')
    for tr in trs:
        elec_dict = otherelec_dict.copy()
        for th in pq(tr)('th'):
            columns.append(pq(th).text().replace(" ","_"))
            add = 0
        for i,td in enumerate(pq(tr)('td')):
            a = pq(td)('a')
            if a:
                elec_dict['elec_link'] = ROOT_LINK+a.attr.href
                elec_dict[columns[i]] = a.text()
            else:
                try:
                    if pq(td)('span') and i < 6:
                        elec_dict[columns[i]] = pq(td)('span').text()
                    else:
                        elec_dict[columns[i]] = str(pq(td).contents()[0]).encode('utf-8').strip().replace(',','')
                except:
                    print ""
                    print "Skipping col %s for %s" % (columns[i], elec_dict['common_link'])
            add = 1
            
        if add == 1:
            all_dicts.append(elec_dict)
    
    return all_dicts

get_otherelec_data(other_link)

[{'Age': '58',
  'Constituency': 'SATNA',
  'Criminal_Cases': 'Yes',
  'Education_Level': 'Post Graduate',
  'Name': 'Ajay Arjun Singh in Loksabha 2014',
  'Number_of_Cases': '2',
  'PAN_Given_(Y_or_N)': 'Y',
  'Party_Code': 'INC',
  'Total_Assets': '254876883',
  'Total_Liabilities': '13845072',
  'common_link': 'http://myneta.info/compare_profile.php?group_id=uRKRoKnCXyy5tSAuuc8b',
  'elec_link': 'http://myneta.info/ls2014/candidate.php?candidate_id=729'},
 {'Age': '58',
  'Constituency': 'CHURHAT',
  'Criminal_Cases': 'Yes',
  'Education_Level': 'Post Graduate',
  'Name': 'Ajay Arjun Singh in Madhya Pradesh 2013',
  'Number_of_Cases': '2',
  'PAN_Given_(Y_or_N)': 'Y',
  'Party_Code': 'INC',
  'Total_Assets': '254586772',
  'Total_Liabilities': '13939632',
  'common_link': 'http://myneta.info/compare_profile.php?group_id=uRKRoKnCXyy5tSAuuc8b',
  'elec_link': 'http://myneta.info/mp2013/candidate.php?candidate_id=258'}]

In [114]:
def get_all_elec():
    all_elec_data = []   
    counter = 0.0
    for key, val in dict_clean.iteritems():
        thelink = get_otherelec_link(val,key)
        counter += 1
        if counter%100 == 0.0:
            print ".",
        if thelink:
            all_elec_data = all_elec_data + get_otherelec_data(thelink)
    
    df = pd.DataFrame(all_elec_data)
    return df.drop_duplicates()

all_elecs_df = get_all_elec()
all_elecs_df.head(3)['common_link']

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Skipping col Constituency for http://myneta.info/compare_profile.php?group_id=h4NLnDQeAZybH8ZvLkvz
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Skipping col Constituency for http://myneta.info/compare_profile.php?group_id=SbP5CVgpok4MGkoZqpN
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Skipping col Constituency for http://myneta.info/compare_profile.php?group

0    http://myneta.info/compare_profile.php?group_i...
1    http://myneta.info/compare_profile.php?group_i...
2    http://myneta.info/compare_profile.php?group_i...
Name: common_link, dtype: object

In [116]:
all_elecs_df.shape

(7793, 12)

In [120]:
all_elecs_df.to_excel("tempdata/all_elecs_statelevel.xls")